In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
filename= "../data/kobe/kobe_bryant_shot_data.csv.gz"
df = pd.read_csv(filename, na_values={'shot_made_flag': ''})
df = df.dropna()
df = df.drop([u'game_event_id', u'game_id',
       u'lat', u'lon', u'team_id', u'team_name', u'game_date',
        u'shot_id'], axis=1)

df = df.drop(['loc_x', 'loc_y', 'shot_type','shot_zone_basic', 'shot_zone_range'], axis=1)

In [3]:
df['home'] = df.matchup.apply(lambda matchup: 0 if '@' in matchup else 1)
df = df.drop(['matchup'], axis=1)

df['time_remaining'] = 60 * df['minutes_remaining'] + df['seconds_remaining']
df = df.drop(['minutes_remaining', 'seconds_remaining'], axis=1)

cols = df.columns.tolist()
cols.remove('shot_made_flag')
cols.append('shot_made_flag')

df = df[cols]

In [4]:
filename= "../data/kobe/kobe_bryant_shot_data_refined.csv"
df.to_csv(filename, index=False)

In [5]:
filename= "../data/kobe/kobe_bryant_shot_data_refined.csv"
df = pd.read_csv(filename)

In [6]:
df.head()

,action_type,combined_shot_type,period,playoffs,season,shot_distance,shot_zone_area,opponent,home,time_remaining,shot_made_flag
0,Jump Shot,Jump Shot,1,0,2000-01,15,Left Side(L),POR,0,622,0.0
1,Jump Shot,Jump Shot,1,0,2000-01,16,Left Side Center(LC),POR,0,465,1.0
2,Jump Shot,Jump Shot,1,0,2000-01,22,Right Side Center(RC),POR,0,412,0.0
3,Driving Dunk Shot,Dunk,2,0,2000-01,0,Center(C),POR,0,379,1.0
4,Jump Shot,Jump Shot,3,0,2000-01,14,Left Side(L),POR,0,572,0.0


In [7]:
original_df = df.copy()

In [8]:
q75 = np.percentile(df.shot_distance, 75)

In [9]:
q75

21.0

In [10]:
df['shot_distance'] = df.shot_distance.clip(upper=45)

In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
scaler = StandardScaler()

In [13]:
df['time_remaining'] = scaler.fit_transform(df.time_remaining.reshape(-1, 1)).reshape(-1, 1)

F:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
F:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [14]:
dist_scaler = StandardScaler()
df['shot_distance'] = dist_scaler.fit_transform(df.shot_distance.reshape(-1, 1)).reshape(-1, 1)

F:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
F:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [15]:
df['shot_zone_area'] = df.shot_zone_area.str.extract('\((\w).*\)')

F:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [16]:
df['period'] = df.period.clip(upper=5)

In [17]:
df.head()

,action_type,combined_shot_type,period,playoffs,season,shot_distance,shot_zone_area,opponent,home,time_remaining,shot_made_flag
0,Jump Shot,Jump Shot,1,0,2000-01,0.170067,L,POR,0,1.442489,0.0
1,Jump Shot,Jump Shot,1,0,2000-01,0.278142,L,POR,0,0.688794,1.0
2,Jump Shot,Jump Shot,1,0,2000-01,0.926591,R,POR,0,0.434362,0.0
3,Driving Dunk Shot,Dunk,2,0,2000-01,-1.451055,C,POR,0,0.275942,1.0
4,Jump Shot,Jump Shot,3,0,2000-01,0.061993,L,POR,0,1.202459,0.0


In [18]:
# turn categorical variables into dummy variables
categorical_vars = ['combined_shot_type', 'season', 'period', 'shot_zone_area', 'opponent', 'action_type']
for var in categorical_vars:
    df = pd.concat([df, pd.get_dummies(df[var], prefix=var)], 1)
    df = df.drop(var, 1)

In [19]:
df.head()

,playoffs,shot_distance,home,time_remaining,shot_made_flag,combined_shot_type_Bank Shot,combined_shot_type_Dunk,combined_shot_type_Hook Shot,combined_shot_type_Jump Shot,combined_shot_type_Layup,...,action_type_Running Tip Shot,action_type_Slam Dunk Shot,action_type_Step Back Jump shot,action_type_Tip Layup Shot,action_type_Tip Shot,action_type_Turnaround Bank shot,action_type_Turnaround Fadeaway shot,action_type_Turnaround Finger Roll Shot,action_type_Turnaround Hook Shot,action_type_Turnaround Jump Shot
0,0,0.170067,0,1.442489,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.278142,0,0.688794,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.926591,0,0.434362,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,-1.451055,0,0.275942,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.061993,0,1.202459,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, log_loss
from sklearn.cross_validation import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('shot_made_flag', axis=1), 
                                                    df['shot_made_flag'], 
                                                    test_size=0.33, 
                                                    random_state=42)

In [22]:
X_train.head()

,playoffs,shot_distance,home,time_remaining,combined_shot_type_Bank Shot,combined_shot_type_Dunk,combined_shot_type_Hook Shot,combined_shot_type_Jump Shot,combined_shot_type_Layup,combined_shot_type_Tip Shot,...,action_type_Running Tip Shot,action_type_Slam Dunk Shot,action_type_Step Back Jump shot,action_type_Tip Layup Shot,action_type_Tip Shot,action_type_Turnaround Bank shot,action_type_Turnaround Fadeaway shot,action_type_Turnaround Finger Roll Shot,action_type_Turnaround Hook Shot,action_type_Turnaround Jump Shot
1119,0,0.386217,0,0.458365,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14464,0,-0.586456,1,-0.482554,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8725,0,0.710441,0,1.850541,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20743,0,-1.451055,0,-1.457077,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24844,1,1.250815,1,-1.533886,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
y_train.head()

1119     1.0
14464    0.0
8725     1.0
20743    0.0
24844    0.0
Name: shot_made_flag, dtype: float64

In [24]:
model = RandomForestClassifier(n_estimators=45, max_depth=14, criterion='entropy', random_state=42, n_jobs=-1)

In [25]:
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=14, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=45, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [26]:
y_pred = model.predict(X_test)

In [27]:
y_pred_proba = model.predict_proba(X_test)

In [28]:
confusion_matrix(y_test, y_pred), log_loss(y_test, y_pred_proba[:,1])

(array([[4030,  668],
        [2052, 1731]]), 0.61180172611564942)

In [ ]:
pd.DataFrame({'feature': X_train.columns, 
              'importance': model.feature_importances_}).sort_values('importance', ascending=False).head()

In [ ]:
pred_df = original_df.join(pd.DataFrame(y_pred, columns=['shot_made_pred'], index=X_test.index))

In [ ]:
pred_df = pred_df[~pred_df.shot_made_pred.isnull()]

In [ ]:
pred_df.head()

In [ ]:
pred_df[(pred_df.shot_made_flag != pred_df.shot_made_pred)]

In [ ]:
# TODO: Other models
# Guess by prev shot? Is it even leagle?
# 'action_type' made a huge diff! Why?

In [ ]:
from sklearn.grid_search import GridSearchCV

param_grid={
    'n_estimators': range(10, 50, 5), 
    'max_depth': range(9, 15),
    'criterion': ['entropy', 'gini'],
}

grid = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs=-1), param_grid=param_grid, scoring='log_loss')

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_score_

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()

In [ ]:
lda.fit(X_train, y_train)

In [ ]:
confusion_matrix(lda.predict(X_test), y_test)

In [ ]:
log_loss(y_test, lda.predict_proba(X_test))

In [37]:
from sklearn.svm import LinearSVC, SVC
svc = SVC(probability=True)

In [38]:
svc.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [39]:
confusion_matrix(svc.predict(X_test), y_test)

array([[4004, 2034],
       [ 694, 1749]])

In [40]:
log_loss(y_test, svc.predict_proba(X_test))

0.62642915215104544

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.svm import LinearSVC, SVC
grid = GridSearchCV(estimator=SVC(), param_grid={})

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_score_